<a href="https://colab.research.google.com/github/vitor1209/-conectados-/blob/main/2025S2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="4"><center>UNIVERSIDADE DE SÃO PAULO</center></font>
<font size="4"><center>INSTITUTO DE CIÊNCIAS MATEMÁTICAS E DE COMPUTAÇÃO</center></font>

<font size="4"><center>Núcleo Interinstitucional de Linguística Computacional (NILC)</center></font>

<br>
<font size="3"><center><b>Retrieval-Augmented Generation: conectando modelos de linguagem a bases de conhecimento</b></center></font>

<font size="3"><center>Prof. Dr. Renato Moraes Silva</center></font>
<br/>

Retrieval-augmented generation (RAG)

Iremos instalar algumas bibliotecas.

In [ ]:
!pip install -U langchain langchain-huggingface langchain-community langchain-text-splitters --quiet

!pip install bs4 --quiet

!pip install pypdf --quiet

!pip install pymupdf4llm --quiet

!pip install sentence-transformers --quiet
!pip install chromadb --quiet

!pip install "unstructured[md]" --quiet





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.

Vamos fazer o download de um PDF: <https://drive.google.com/file/d/1yFPI0BrK6nrjnP8PspjI9D_bFfIot9M4/view?usp=sharing>

In [ ]:
import os
import requests

def download(url, save_path):

    # Cria a pasta se não existir
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    print("Baixando...")
    response = requests.get(url)

    if response.status_code != 200:
        print("Erro ao baixar:", response.status_code)
        return

    # Salva o conteúdo diretamente em arquivo
    with open(save_path, "wb") as f:
        f.write(response.content)

    print(f"Download concluído. Arquivo salvo em: {save_path}")

url = 'https://drive.google.com/uc?export=download&id=1yFPI0BrK6nrjnP8PspjI9D_bFfIot9M4'
pathFile = 'dados/vale.pdf'
download(url, pathFile)

Baixando...
Download concluído. Arquivo salvo em: dados/vale.pdf


Lendo o arquivo.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

########################## COMPLETE O CÓDIGO AQUI  ########################

loader = PyPDFLoader(pathFile)
docs = loader.load()

for i, pagina in enumerate(docs[1:2]):
  print(f"Página {i+1}: {pagina.page_content}")

###########################################################################

Página 1: – 2 –  
 
Destaques 
dos 
Negócios 
 
Soluções de Minério de Ferro 
• O primeiro embarque do projeto Capanema foi concluído no 2T . O projeto adicionará 15 Mtpa de 
capacidade de produção líquida de minério de ferro, com a conclusão do ramp-up esperado para o primeiro 
semestre de 2026. Essa expansão contribuirá para o atingimento do guidance de produção de 2025 (325-
335 Mt). 
Metais para Transição Energética 
• A Vale Metais Básicos obteve a Licença Prévia para o projeto Bacaba em junho . O projeto tem como 
objetivo estender a vida útil do Complexo Minerador de Sossego, contribuindo com uma produção média 
anual de cobre de aproximadamente 50 ktpa ao longo de 8 anos de operação. Aproximadamente US$ 290 
milhões serão investidos durante a fase de implantação do projeto e o início da produção está planejado 
para o primeiro semestre de 2028. 
• O comissionamento do projeto do 2º forno de Onça Puma iniciou em julho . Com um investimento 
planejado de US$ 555 milhões, o projet

In [ ]:
for i, pagina in enumerate(docs[3:5]):
  print(f"Página {i+1}: {pagina.page_content}")

Página 1: Resultado Financeiro 
 
 
 – 4 –  
 
EBITDA 
EBITDA Proforma totalizou US$ 3,4 bilhões no 2T25, 14% menor a/a, devido principalmente a queda de 13% nos preços de 
referência do minério de ferro. Esse efeito foi parcialmente compensado por (i) avanços contínuos nas iniciativas de eficiência de 
custos e despesas, que impulsionaram a melhoria de desempenho nos segmentos de Metais para Transição Energética e Minério de 
Ferro, e (ii) efeito positivo da depreciação do real. 
EBITDA Proforma 2T25 vs. 2T24 - US$ milhões 
 
1 Excluindo despesas de Brumadinho. O EBITDA Proforma do 2T24 foi reapresentado excluindo eventos não recorrentes (US$ 5 milhões). 2 Incluindo volume, EBITDA de coligadas e JV's, e outros. 
 
 
 
 
 
Lucro Líquido 
O lucro líquido proforma foi de US$ 2,1 bilhões no 2T25 , 6% maior a/a, impactado positivamente pela marcação a mercado 
de swaps, que se beneficiaram de movimentos cambiais favoráveis ao longo do trimestre, contribuindo para o aumento de US$ 
1,4 bilh

Usando um leitor melhor.

In [ ]:
import pymupdf4llm  # precisa instalar: pip install pymupdf4llm

md_path = os.path.splitext(pathFile)[0] + ".md"

########################## COMPLETE O CÓDIGO AQUI  ########################

md_text = pymupdf4llm.to_markdown(pathFile)

###########################################################################

with open(md_path, "w", encoding="utf-8") as f:
    f.write(md_text)
    print(f"Convertido: {pathFile} -> {md_path}")


Convertido: dados/vale.pdf -> dados/vale.md


Vamos carregar um arquivo markdown.

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

########################## COMPLETE O CÓDIGO AQUI  ########################

loader = UnstructuredMarkdownLoader(md_path)
docs = loader.load()

for i, pagina in enumerate(docs[0:2]):
  print(f"Página {i+1}: {pagina.page_content}")

###########################################################################


Página 1: US$milhões 2T25 2T24 Δa/a 1T25 Δt/t 6M25 6M24 Δa/a Receita líquida de vendas 8.804 9.920 -11 % 8.119 8 % 16.923 18.379 -8 % 9.920 -11 % 8.119 8 % 16.923 18.379 -8 % 9.920 -11 % 8.119 8 % 16.923 18.379 -8 % 9.920 -11 % 8.119 8 % 16.923 18.379 -8 % Custos e despesas totais (ex-Brumadinho e descaracterização de barragens)¹ (6.462) (6.974) -7 % (5.803) 11 % (12.265) (12.871) -5 % (6.974) -7 % (5.803) 11 % (12.265) (12.871) -5 % (6.974) -7 % (5.803) 11 % (12.265) (12.871) -5 % (6.974) -7 % (5.803) 11 % (12.265) (12.871) -5 % Despesas relacionadas a Brumadinho e descaracterização de barragens (38) 1 n.a. (97) -61 % (135) (40) 238 % 1 n.a. (97) -61 % (135) (40) 238 % 1 n.a. (97) -61 % (135) (40) 238 % 1 n.a. (97) -61 % (135) (40) 238 % EBIT ajustado 2.606 3.200 -19 % 2.411 8 % 5.017 5.924 -15 % 3.200 -19 % 2.411 8 % 5.017 5.924 -15 % 3.200 -19 % 2.411 8 % 5.017 5.924 -15 % 3.200 -19 % 2.411 8 % 5.017 5.924 -15 % EBITDA ajustado 3.386 3.993 -15 % 3.115 9 % 6.501 7.431 -13 % 3.993 -15

Usando um banco de dados vetorial

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Cria documentos com IDs únicos e metadados
documentos = [
    "O gato dorme no sofá.",
    "O cachorro corre no parque.",
    "O sol nasce no leste e se põe no oeste.",
    "A chuva veio muito forte.",
    "A inteligência artificial está transformando o mundo."
]

metadados = [
    {"categoria": "animais", "idioma": "pt"},
    {"categoria": "animais", "idioma": "pt"},
    {"categoria": "natureza", "idioma": "pt"},
    {"categoria": "natureza", "idioma": "pt"},
    {"categoria": "tecnologia", "idioma": "pt"}
]

########################## COMPLETE O CÓDIGO AQUI  ########################










###########################################################################

print("\n=== Resultados da busca ===")
for doc, meta, dist in zip(resultados['documents'][0],resultados['metadatas'][0],resultados['distances'][0]):
    print(f"- {doc} (categoria={meta['categoria']}, distância={dist:.4f})")

Vamos carregar um documento Web usando Langchain.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


url = "https://www.icmc.usp.br/noticias/7268-arvores-que-falam-inscreva-se-na-atividade-de-extensao-do-museu-da-fauna-e-flora-do-icmc"

def get_texto(url):

############################# COMPLETE O CÓDIGO AQUI  ########################









##############################################################################

data = get_texto(url)

display(data)

Criando uma pequena interface no Gradio para obter os dados.

In [ ]:
import gradio as gr

# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🔎 Chat RAG com Gradio")

    pergunta_input = gr.Textbox(
        label="Digite sua pergunta:",
        placeholder="Ex: Explique como será a seleção."
    )

    resposta_output = gr.Textbox(
        label="Resposta",
        lines=10
    )

    botao = gr.Button("Enviar")

    # Quando clicar no botão, chama sua função
    botao.click(
        fn=get_texto,
        inputs=pergunta_input,
        outputs=resposta_output
    )


# Testa a interface
demo.launch()


Iremos dividir o texto em pequenos pedaços, chamados de *chunks*.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

############################# COMPLETE O CÓDIGO AQUI  ########################








##############################################################################


Iremos criar embeddings para cada pedaço de texto.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings_modelName = 'rufimelo/bert-large-portuguese-cased-sts'

############################# COMPLETE O CÓDIGO AQUI  ########################









##############################################################################

print(retriever)


Iremos testar as embeddings.

In [ ]:
pergunta = "Quem serão os estudantes selecionados"

############################# COMPLETE O CÓDIGO AQUI  ########################








##############################################################################


Usando Faiss

In [ ]:
# pip install faiss-gpu
!pip install faiss-cpu --quiet

import numpy as np
import faiss


vetores = np.array([
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.1, 0.9, 0.8],
    [0.9, 0.8, 0.2, 0.1],
    [0.4, 0.3, 0.2, 0.1]
], dtype='float32')

############################# COMPLETE O CÓDIGO AQUI  ########################








##############################################################################

print("\n=== Resultados da busca básica ===")
for i in range(k):
    print(f"Mais similar {i+1}: índice {indices[0][i]} (distância = {distancias[0][i]:.4f})")


A busca básica possui alto custo computacional. Use busca baseada em clusters.

In [ ]:
print("\n=== Busca estruturada: IndexIVFFlat ===")

# Parâmetros
d = 4          # dimensão dos vetores
nb = 1000      # número de vetores base
nq = 5         # número de consultas
nlist = 10     # número de clusters no IVF
nprobe = 3     # quantos clusters buscar

# Gera dados aleatórios
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')

############################# COMPLETE O CÓDIGO AQUI  ########################












##############################################################################

print("Distâncias:", D)
print("Índices:", I)

Para mais eficiência, é possivel usar quantização também.

In [ ]:
m = 2  # número de subvetores
nbits = 8  # bits por subvetor

############################# COMPLETE O CÓDIGO AQUI  ########################









##############################################################################

print("Top 3 resultados (PQ):", indices_pq)
print("Distâncias (PQ):", distancias_pq)

# salva o índice
faiss.write_index(index_flat, "dados/meu_indice_flat.faiss")

# carrega o índice
index_carregado = faiss.read_index("dados/meu_indice_flat.faiss")
print("\nÍndice recarregado contém", index_carregado.ntotal, "vetores.")

Usando Grafos

In [ ]:
# !pip install networkx langchain

import networkx as nx
from langchain_core.documents import Document

# Cria um grafo
G = nx.Graph()

# Adicionar nós (entidades)
G.add_nodes_from([
    ("Covid-19", {"type": "doença"}),
    ("Vacina", {"type": "intervenção"}),
    ("São Paulo", {"type": "local"}),
    ("Hospital das Clínicas", {"type": "instituição"}),
    ("Gripe", {"type": "doença"}),
])

# Adicionar arestas (relações)
G.add_edges_from([
    ("Covid-19", "Vacina", {"relation": "tratamento"}),
    ("Covid-19", "São Paulo", {"relation": "ocorrência"}),
    ("Vacina", "Hospital das Clínicas", {"relation": "aplicação"}),
    ("Gripe", "Vacina", {"relation": "tratamento"}),
])

query = "Qual é a relação entre vacinação contra Covid-19 e hospitais em São Paulo?"

# Recuperar subgrafo relevante para a query
# Selecionar nós que aparecem na query
relevant_nodes = [node for node in G.nodes if node.lower() in query.lower()]
subgraph = G.subgraph(relevant_nodes)

# Criar representação textual do subgrafo
subgraph_text = ""
for u, v, data in G.edges(data=True):
    if u in subgraph.nodes or v in subgraph.nodes:
        subgraph_text += f"{u} --{data['relation']}--> {v}\n"

############################# COMPLETE O CÓDIGO AQUI  ########################








##############################################################################

# Mostrar a entrada que seria passada para o LLM
print("Entrada para LLM (simulada):\n")
for doc in documents:
    print("-", doc.page_content)


Calculando a vizinhança de forma mais eficiente

In [ ]:
# Identificar nós iniciais da query
seed_nodes = [node for node in G.nodes if node.lower() in query.lower()]

k = 2  # número de hops
nodes_to_include = set(seed_nodes)

for node in seed_nodes:
    for hop in range(1, k+1):
        # Adiciona vizinhos até k hops
        neighbors = nx.single_source_shortest_path_length(G, node, cutoff=hop)
        nodes_to_include.update(neighbors.keys())

subgraph = G.subgraph(nodes_to_include)

# Converter para texto
subgraph_text = ""
for u, v, data in G.edges(data=True):
    if u in subgraph.nodes or v in subgraph.nodes:
        subgraph_text += f"{u} --{data['relation']}--> {v}\n"

# Criar documentos do LangChain
documents = [Document(page_content=line.strip()) for line in subgraph_text.split("\n") if line]

# Mostrar a entrada que seria passada para o LLM
print("Entrada para LLM (simulada):\n")
for doc in documents:
    print("-", doc.page_content)

Teste final: <https://drive.google.com/file/d/1pyQzUO7CMJnXZRMs8od4Fa21myoBSUs2/view?usp=drive_link>